In [66]:
import pandas as pd
import zipfile
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

In [41]:
# -----------------------------------------------------------
# Load Data
# -----------------------------------------------------------
zf = zipfile.ZipFile('digit-recognizer.zip') 
train = pd.read_csv(zf.open('train.csv'))
test = pd.read_csv(zf.open('test.csv'))

In [42]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# -----------------------------------------------------------
# Define train data
# -----------------------------------------------------------
x_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"]
y_train = to_categorical(y_train, num_classes = 10)

In [44]:
# -----------------------------------------------------------
# Reshaping image to be fed to Neural Network
# -----------------------------------------------------------
x_train = x_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# -----------------------------------------------------------
# Normalize image values
# -----------------------------------------------------------
x_train = x_train / 255
test = test / 255

In [45]:
# -----------------------------------------------------------
# Set the random seed
# -----------------------------------------------------------
random_seed = 1

# -----------------------------------------------------------
# Split the train and the validation set
# -----------------------------------------------------------
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=random_seed)

In [72]:
# -----------------------------------------------------------
# Using Image Augmentation to regularize and improve training
# -----------------------------------------------------------
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.2,
        width_shift_range=0.2,
        height_shift_range=0.2)  


datagen.fit(x_train)

In [ ]:
# -----------------------------------------------------------
# Convolutional Neural Network
# Build
# -----------------------------------------------------------
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# -----------------------------------------------------------
# Saving model using Checkpoint method
# -----------------------------------------------------------
filepath="best_model_file.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

# -----------------------------------------------------------
# Compile
# -----------------------------------------------------------
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

# -----------------------------------------------------------
# Train
# -----------------------------------------------------------
history = model.fit(datagen.flow(x_train, y_train, batch_size=256),
                    batch_size=256,
                    epochs=15,
                    verbose=1,
                    validation_data=(x_val,y_val),
                    callbacks=[checkpoint]
                   )

Epoch 1/15
148/148 [==============================] - ETA: 0s - loss: 0.7662 - accuracy: 0.7448
Epoch 00001: val_loss improved from inf to 0.12011, saving model to best_model_file.h5
148/148 [==============================] - 38s 257ms/step - loss: 0.7662 - accuracy: 0.7448 - val_loss: 0.1201 - val_accuracy: 0.9600
Epoch 2/15
148/148 [==============================] - ETA: 0s - loss: 0.2168 - accuracy: 0.9342
Epoch 00002: val_loss improved from 0.12011 to 0.06081, saving model to best_model_file.h5
148/148 [==============================] - 37s 253ms/step - loss: 0.2168 - accuracy: 0.9342 - val_loss: 0.0608 - val_accuracy: 0.9814
Epoch 3/15
148/148 [==============================] - ETA: 0s - loss: 0.1508 - accuracy: 0.9541
Epoch 00003: val_loss did not improve from 0.06081
148/148 [==============================] - 37s 253ms/step - loss: 0.1508 - accuracy: 0.9541 - val_loss: 0.0848 - val_accuracy: 0.9721
Epoch 4/15
148/148 [==============================] - ETA: 0s - loss: 0.1134 - ac

In [ ]:
# -----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
# -----------------------------------------------------------
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))  # Get number of epochs

# ------------------------------------------------
# Plot training and validation accuracy per epoch
# ------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

# ------------------------------------------------
# Plot training and validation loss per epoch
# ------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")

plt.title('Training and validation loss')

In [ ]:
# -----------------------------------------------------------
# Load Best Model file
# -----------------------------------------------------------
 model = tensorflow.keras.models.load_model('best_model_file.h5')

In [ ]:
# -----------------------------------------------------------
# Predict on Unseen Data
# -----------------------------------------------------------
predictions = model.predict_classes(test, verbose=0)

submissions = pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)), "Label": predictions})
submissions.to_csv("digit_recognizer_submission.csv", index=False, header=True)